In [ ]:
import sys
print(sys.version)

3.10.12 (main, Nov 20 2023, 15:14:05) [GCC 11.4.0]


In [15]:
!python --version

Python 3.10.12


In [6]:
import numpy as np
from google.colab import drive
drive.mount('/content/drive')

dataset_path = '/content/drive/MyDrive/data/'

train_images = np.load(dataset_path + 'train_images.npy')
train_labels = np.load(dataset_path + 'train_labels.npy')
test_images = np.load(dataset_path + 'test_images.npy')
test_labels = np.load(dataset_path + 'test_labels.npy')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
print("Train Images Shape:", train_images.shape)
print("Train Labels Shape:", train_labels.shape)
print("Test Images Shape:", test_images.shape)
print("Test Labels Shape:", test_labels.shape)

print("\nTrain Images Data Type:", train_images.dtype)
print("Train Labels Data Type:", train_labels.dtype)
print("Test Images Data Type:", test_images.dtype)
print("Test Labels Data Type:", test_labels.dtype)

print("\nTrain Images Min Value:", np.min(train_images))
print("Train Images Max Value:", np.max(train_images))
print("Test Images Min Value:", np.min(test_images))
print("Test Images Max Value:", np.max(test_images))

print("\nUnique Train Labels:", np.unique(train_labels))
print("Unique Test Labels:", np.unique(test_labels))

Train Images Shape: (28754, 48, 48)
Train Labels Shape: (28754, 7)
Test Images Shape: (7178, 48, 48)
Test Labels Shape: (7178, 7)

Train Images Data Type: float64
Train Labels Data Type: float32
Test Images Data Type: float64
Test Labels Data Type: float32

Train Images Min Value: 0.0
Train Images Max Value: 1.0
Test Images Min Value: 0.0
Test Images Max Value: 1.0

Unique Train Labels: [0. 1.]
Unique Test Labels: [0. 1.]


In [8]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator


input_shape = (48, 48, 1)

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=input_shape),
    BatchNormalization(),
    Conv2D(32, (3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D(2, 2),

    Conv2D(64, (3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    Conv2D(64, (3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D(2, 2),

    Conv2D(128, (3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    Conv2D(128, (3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D(2, 2),

    Conv2D(256, (3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    Conv2D(256, (3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    GlobalAveragePooling2D(),

    Dense(256, activation='relu'),
    BatchNormalization(),
    Dropout(0.3),
    Dense(7, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

epochs = 40
batch_size = 32

#Data augmentation parameters
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

train_images = train_images.reshape((-1, 48, 48, 1))
test_images = test_images.reshape((-1, 48, 48, 1))

train_generator = datagen.flow(
    train_images,
    train_labels,
    batch_size=batch_size
)

model.fit(
    train_generator,
    steps_per_epoch=len(train_images) // batch_size,
    epochs=epochs,
    validation_data=(test_images, test_labels)
)

loss, accuracy = model.evaluate(test_images, test_labels)
print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_16 (Conv2D)          (None, 48, 48, 32)        320       
                                                                 
 batch_normalization_17 (Ba  (None, 48, 48, 32)        128       
 tchNormalization)                                               
                                                                 
 conv2d_17 (Conv2D)          (None, 48, 48, 32)        9248      
                                                                 
 batch_normalization_18 (Ba  (None, 48, 48, 32)        128       
 tchNormalization)                                               
                                                                 
 max_pooling2d_6 (MaxPoolin  (None, 24, 24, 32)        0         
 g2D)                                                            
                                                      

In [9]:
model.save('emotion_detection_model.h5')

converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with open('emotion_detection_model.tflite', 'wb') as f:
    f.write(tflite_model)

from google.colab import files

files.download('emotion_detection_model.h5')
files.download('emotion_detection_model.tflite')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>